In [1]:
import pandas as pd

In [2]:
# df = pd.read_csv('https://s3.amazonaws.com/techblog-static/interview_dataset.csv')
# df.to_pickle('okc_df.pkl')
df = pd.read_pickle('okc_df.pkl')

In [ ]:
#https://brycexxx.github.io/2018/11/15/c4-assignment3/

In [3]:
df.shape

(100000, 6)

In [4]:
df[df.isna().any(axis=1)]

,Unnamed: 0,Body,Title,label,Body_processed,Title_processed
26633,74726,<p>script async not working:</p>\n\n<pre><code...,<script async> not working in rails,0,script async not working : < h1 > welcome # in...,NaN
90392,65968,<p>In my JSF+Spring project i tried to add for...,<rich:popupPanel buttons not working in JSF Pr...,0,in my jsf + spring project i tried to add form...,NaN


Note that the only row is missing is where the preprocessor failed. Since these are only two rows, we drop.

In [5]:
df.dropna(inplace=True)

In [6]:
df[df.label==1].iloc[0:5]

,Unnamed: 0,Body,Title,label,Body_processed,Title_processed
0,33226,<p>Hi I'm new to <code>Ruby on Rails</code>. I...,RUBY: most common number for Users,1,hi i'm new to ruby on rails . i created users ...,ruby : most common number for users
2,39763,<p>I have a search engine on PHP that have ind...,Scan a webpage and get the video embed url only,1,i have a search engine on php that have indexe...,scan a webpage and get the video embed url only
4,9698,<p>For my website I want to have divs out of t...,Use jQuery or Javascript to move elements into...,1,for my website i want to have divs out of the ...,use jquery or javascript to move elements into...
5,5948,<p>I'm looking for a dead simple Java Library ...,Java SFTP Transfer Library,1,i'm looking for a dead simple java library to ...,java sftp transfer library
6,27955,<p>Having two MySql databases with same struct...,How to Merge two Mysql Databases with same str...,1,having two mysql databases with same structure...,how to merge two mysql databases with same str...


In [7]:
df[df.label==0].iloc[0:5]

,Unnamed: 0,Body,Title,label,Body_processed,Title_processed
1,64804,<p>I know that StringBuffer class is synchroni...,What exactly does it mean when they say that S...,0,i know that stringbuffer class is synchronized...,what exactly does it mean when they say that s...
3,51270,<p>Is there any way to get ICY metadata from s...,ICY metadata support with ffmpeg,0,is there any way to get icy metadata from shou...,icy metadata support with ffmpeg
7,55001,<p>Hi I have 3 soap requests. \nSay.\nReq1 \nR...,How can I pipeline multiple soap requests usin...,0,hi i have 3 soap requests . say . req1 req2 re...,how can i pipeline multiple soap requests usin...
8,50875,<p>Consider the following entity-relationship ...,Mapping many-to-many relationship with attribu...,0,consider the following entity - relationship d...,mapping many - to - many relationship with att...
14,64298,"<p>I am trying to strip all ""?"" in file names ...",Recursively rename directories and files based...,0,"i am trying to strip all ""?"" in file names in ...",recursively rename directories and files based...


In [8]:
df['Body_length'] = df.Body_processed.apply(lambda x: len(x.split()))

In [9]:
df['Body_length'].mean()

210.8884877697554

In [10]:
print(df[df['label'] == 0]['Body_length'].mean())
print(df[df['label'] == 1]['Body_length'].mean())

284.05628225129004
137.72362


In [11]:
df['Title_length'] = df.Title_processed.apply(lambda x: len(x.split()))

In [12]:
df['Title_length'].mean()

9.41084821696434

In [13]:
print(df[df['label'] == 0]['Title_length'].mean())
print(df[df['label'] == 1]['Title_length'].mean())

9.722528901156046
9.09918


In [14]:
# get number of unique words

In [15]:
results = set()
df[df['label'] == 1]['Body_processed'].str.lower().str.split().apply(results.update)
print(len(list(results)))


161065


In [16]:
results = set()
df[df['label'] == 0]['Body_processed'].str.lower().str.split().apply(results.update)
print(len(list(results)))


354779


In [17]:
# possibly : add the 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Title_processed'] + df['Body_processed'], 
                                                    df['label'], 
                                                    random_state=0)
def answer_two():
    vect = CountVectorizer().fit(X_train)
    tokens = vect.get_feature_names()
    return sorted(tokens, key=len)

In [19]:
test = answer_two()

In [20]:
test[50000:50010]

['marié',
 'mark1',
 'mark2',
 'mark3',
 'marki',
 'marko',
 'marks',
 'marry',
 'marsh',
 'marst']

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    feature_names = np.array(vect.get_feature_names()).reshape(-1, 1)
    X_train_vectorized = vect.transform(X_train)
    tfidf_values = X_train_vectorized.max(0).toarray()[0].reshape(-1, 1)
    tfidf_df = pd.DataFrame(data=np.hstack((feature_names, tfidf_values)), columns=['features', 'tfidf'])
    smallest_tfidf = tfidf_df.sort_values(by=['tfidf', 'features']).set_index('features')[:20]
    largest_tfidf = tfidf_df.sort_values(by=['tfidf', 'features'], ascending=[False, True]).set_index('features')[:20]
    return smallest_tfidf['tfidf'].apply(float), largest_tfidf['tfidf'].apply(float)


In [22]:
answer_four()

(features
 0303                                           0.001006
 06011982                                       0.001006
 0863                                           0.001006
 805917662                                      0.001006
 agl                                            0.001006
 bankruptcies                                   0.001006
 businessdecisionresultoverride                 0.001006
 createnewoutputrows                            0.001006
 dcqjwzg                                        0.001006
 engy                                           0.001006
 getcacheentryrspxml                            0.001006
 individualalerts                               0.001006
 individualcommercialdecision                   0.001006
 judgements                                     0.001006
 mktg                                           0.001006
 proprietorships                                0.001006
 summons                                        0.001006
 vedascorefinancialco

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    clf = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    y_score = clf.predict_proba(X_test_vectorized)[:, 1]
    score = roc_auc_score(y_test, y_score)
    return score

In [24]:
answer_five()

0.84107187564574

In [25]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [26]:

def answer_seven2():
    from scipy.sparse import csr_matrix, hstack
    temp = df.copy()
    temp['length_of_doc'] = df['Body_processed'].str.len()
    X_train, X_test, y_train, y_test = train_test_split(temp.drop('label', axis=1),
                                                        temp['label'] , random_state=0)
    vect = TfidfVectorizer(min_df=5).fit(X_train['Body_processed'])
    
    X_train_vectorized_body = vect.transform(X_train['Body_processed'])
    X_train_vectorized_title = vect.transform(X_train['Title_processed'])
    X_train_vectorized = hstack([X_train_vectorized_body, X_train_vectorized_title],
                                'csr')
    
    clf = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    
    X_test_vectorized_body = vect.transform(X_test['Body_processed'])  
    X_test_vectorized_title = vect.transform(X_test['Title_processed'])
    X_test_vectorized = hstack([X_test_vectorized_body, X_test_vectorized_title],
                                'csr')
    
    y_score = clf.predict_proba(X_test_vectorized)[:, 1]
    score = roc_auc_score(y_test, y_score)

    return score

In [27]:
answer_seven2() # increased slightly by keeping title & body separate.

0.8482461288589771

In [28]:
from sklearn.svm import SVC

def answer_seven():
    temp = df.copy()
    temp['length_of_doc'] = df['Body_processed'].str.len()
    X_train, X_test, y_train, y_test = train_test_split(temp.drop('label', axis=1),
                                                        temp['label'] , random_state=0)
    vect = TfidfVectorizer(min_df=5).fit(X_train['Body_processed'])
    X_train_vectorized = vect.transform(X_train['Body_processed'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['length_of_doc'])
    clf = SVC(C=10000).fit(X_train_vectorized, y_train)
    X_test_vectorized = vect.transform(X_test['text'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['length_of_doc'])
    y_score = clf.decision_function(X_test_vectorized)
    score = roc_auc_score(y_test, y_score)
    return score

In [29]:
#answer_seven()

In [60]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
import nltk
import re
from xgboost import XGBClassifier

def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words


Pipeline([('colext', TextSelector('Body_processed')),
          ('tfidf', TfidfVectorizer(tokenizer=Tokenizer,
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
          ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])

Pipeline(memory=None,
         steps=[('colext', TextSelector(field='Body_processed')),
                ('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.25, max_features=None,
                                 min_df=0.0025, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function Tokenizer at 0x1267e5700>,
                                 use_idf=True, vocabulary=None)),
                ('svd',
                 TruncatedSVD(algorithm='ran

In [40]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [63]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('Text')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, #stop_words=stop_words,
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=100)), #for XGB
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('Body_length')),
            ('wscaler', StandardScaler()),
        ])),
    ])),
    ('clf', XGBClassifier(max_depth=3, n_estimators=100, learning_rate=0.1)),
    ])

In [64]:
df.columns

Index(['Unnamed: 0', 'Body', 'Title', 'label', 'Body_processed',
       'Title_processed', 'Body_length', 'Title_length', 'comb', 'Text'],
      dtype='object')

In [65]:
df['Text'] = df['Body_processed'] + df['Title_processed']

In [66]:
df.columns

Index(['Unnamed: 0', 'Body', 'Title', 'label', 'Body_processed',
       'Title_processed', 'Body_length', 'Title_length', 'comb', 'Text'],
      dtype='object')

In [67]:
X = df[['Text', 'Body_length']]
y = df['label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)


In [ ]:
#https://medium.com/@kocur4d/hyper-parameter-tuning-with-pipelines-5310aff069d6